In [2]:
import tensorflow as tf
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

Dataset= pd.read_csv("Training.csv")


Dataset=Dataset.dropna() # remove null values
Dataset=Dataset.drop("Id",axis=1) # remove sample information
#print(Dataset.value_counts(Dataset["Class"]))
Labels=Dataset[["Class"]].copy() # generate separate class dataframe
Dataset=Dataset.drop("Class",axis=1) # drop class from feature dataframe
mapping = {'A': 1, 'B': 0} # change object from str to int 
Dataset['EJ'] = Dataset['EJ'].replace(mapping)
#print(Dataset.shape)
Dataset_train, Dataset_test, Labels_train, Labels_test = train_test_split(Dataset, Labels, test_size=0.5, random_state=42)
#print(type(Dataset_train))
sm = SMOTE(random_state=42)
AltDataset, AltLabels = sm.fit_resample(Dataset_train, Labels_train)
#print(AltLabels.value_counts(AltLabels["Class"]))
AltDataset = tf.convert_to_tensor(AltDataset.values, dtype=tf.float32)
AltLabels=tf.convert_to_tensor(AltLabels.values, dtype=tf.float32)
print(len(Labels_train))
DiseaseModel=keras.Sequential([layers.Dense(300,activation="relu"),layers.Dense(1,activation="sigmoid")])
DiseaseModel.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
DiseaseModel.fit(AltDataset,AltLabels,epochs=30,batch_size=150)
print("I will determine the accuracy of my test data.")
Evaluations=DiseaseModel.evaluate(Dataset_test,Labels_test)

274
Epoch 1/30
3/3 [==============================] - 1s 6ms/step - loss: 746.9918 - accuracy: 0.4729
Epoch 2/30
3/3 [==============================] - 0s 4ms/step - loss: 320.4411 - accuracy: 0.4299
Epoch 3/30
3/3 [==============================] - 0s 3ms/step - loss: 190.0853 - accuracy: 0.5701
Epoch 4/30
3/3 [==============================] - 0s 3ms/step - loss: 153.3307 - accuracy: 0.5769
Epoch 5/30
3/3 [==============================] - 0s 4ms/step - loss: 205.1470 - accuracy: 0.5973
Epoch 6/30
3/3 [==============================] - 0s 4ms/step - loss: 157.7244 - accuracy: 0.5995
Epoch 7/30
3/3 [==============================] - 0s 4ms/step - loss: 102.5309 - accuracy: 0.6176
Epoch 8/30
3/3 [==============================] - 0s 4ms/step - loss: 86.2300 - accuracy: 0.6018
Epoch 9/30
3/3 [==============================] - 0s 3ms/step - loss: 63.1955 - accuracy: 0.6063
Epoch 10/30
3/3 [==============================] - 0s 4ms/step - loss: 69.7135 - accuracy: 0.5973
Epoch 11/30
3/3 [=

In [40]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

def create_model():
    model = Sequential([Dense(150, activation="relu"), Dense(1, activation="sigmoid")])
    return model

loss_function = tf.keras.losses.BinaryCrossentropy()

num_models = 100
models = [create_model() for _ in range(num_models)]  # Initialize models


for generation in range(min(50, len(AltDataset))):  # For each generation
    losses = []

    for model in models:  # For each model
        
        sample, label = AltDataset[generation], AltLabels[generation]
        optimizer = tf.keras.optimizers.Adam()  # Define optimizer inside the loop
        
        # Train the model on the sample
        with tf.GradientTape() as tape:
            prediction = model(sample[None, ...], training=True)  # Forward pass
            loss = loss_function(label[None, ...], prediction)  # Compute the loss

        gradients = tape.gradient(loss, model.trainable_variables)  # Compute the gradients
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))  # Update the weights

        losses.append(loss)

    sorted_indices = (np.argsort(losses))[1:int((len(models)/2)+1)]
    top_20_idx = sorted_indices[:len(models) * 20 // 100]
    mid_40_idx = sorted_indices[len(models) * 20 // 100:len(models) * 60 // 100]
    bottom_40_idx = sorted_indices[len(models) * 60 // 100:]

    def crossover_and_create_models(parents, num_children_per_pair):
        new_models = []
        num_pairs = len(parents) // 2 * 2  # Ensures an even number of parents
        for i in range(0, num_pairs, 2):  # We'll use pairs of parents
            for _ in range(num_children_per_pair):
                new_model = create_model()
                new_model.build((None, sample.shape[0]))
                for j in range(len(new_model.weights)):
                    mask = np.random.choice([0, 1], size=new_model.weights[j].shape.as_list())
                    new_weights = np.where(mask, models[parents[i]].weights[j].numpy(), models[parents[i + 1]].weights[j].numpy())
                    new_model.weights[j].assign(new_weights)
                new_models.append(new_model)
                total_weight = sum([np.sum(w.numpy()) for w in new_model.weights])
                #print(f"Sum of all weights for model {_}: {total_weight}")
        return new_models


    # Creating children from top 20%, mid 40%, and bottom 40%
    #new_models_top_20 = crossover_and_create_models(top_20_idx, 3)
    #new_models_mid_40 = crossover_and_create_models(mid_40_idx, 2)
    #new_models_bottom_40 = crossover_and_create_models(bottom_40_idx, 1)
    new_models=crossover_and_create_models(sorted_indices,4)
    # Combine all the newly created models
    #new_models = new_models_top_20 + new_models_mid_40 + new_models_bottom_40
    

    models = new_models
    


In [42]:
print(models)
from tensorflow.keras.metrics import BinaryAccuracy

for i, model in enumerate(models):  
      y_pred = model.predict(Dataset_test)  # Assuming Dataset_test is your test data
      y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to class labels (0 or 1)

      conf_matrix = np.zeros((2, 2), dtype=int)
      conf_matrix[0, 0] = np.sum((Labels_test.values == 0) & (y_pred == 0))  # True Negatives (TN)
      conf_matrix[0, 1] = np.sum((Labels_test.values == 0) & (y_pred == 1))  # False Positives (FP)
      conf_matrix[1, 0] = np.sum((Labels_test.values == 1) & (y_pred == 0))  # False Negatives (FN)
      conf_matrix[1, 1] = np.sum((Labels_test.values == 1) & (y_pred == 1))  # True Positives (TP)

      # Extract true negatives (TN), false positives (FP), true positives (TP), and false negatives (FN) from the confusion matrix
      TN, FP, TP, FN = conf_matrix[0, 0], conf_matrix[0, 1], conf_matrix[1, 1], conf_matrix[1, 0]

      # Calculate specificity, sensitivity, precision, NPV, and accuracy
      specificity = TN / (TN + FP)
      sensitivity = TP / (TP + FN)
      precision_class_one = TP / (TP + FP)
      Negative_Predictive_Value = TN / (TN + FN)
      accuracy = (TP + TN) / (TP + TN + FP + FN)

      # Print the metrics
      """
      print(f"Model {i+1}")
      print(f"Sensitivity: {sensitivity * 100:.2f}%,",
            f"Specificity: {specificity * 100:.2f}%,",
            f"Accuracy: {accuracy * 100:.2f}%,",
            f"Negative Predictive Value: {Negative_Predictive_Value * 100:.2f}%,",
            f"Precision (Class One): {precision_class_one * 100:.2f}%")
      print(f"True Negatives (TN): {TN},",
            f"False Positives (FP): {FP},",
            f"True Positives (TP): {TP},",
            f"False Negatives (FN): {FN}")
      """

      if sensitivity > 0.4 and specificity >0.8:
            print("Green")
            print(f"Model {i+1}")
            print(f"Sensitivity: {sensitivity * 100:.2f}%,",
                  f"Specificity: {specificity * 100:.2f}%,",
                  f"Accuracy: {accuracy * 100:.2f}%,",
                  f"Negative Predictive Value: {Negative_Predictive_Value * 100:.2f}%,",
                  f"Precision (Class One): {precision_class_one * 100:.2f}%")
            print(f"True Negatives (TN): {TN},",
                  f"False Positives (FP): {FP},",
                  f"True Positives (TP): {TP},",
                  f"False Negatives (FN): {FN}")

Proportion = Labels_test["Class"].value_counts()
print("By random chance the precision would be {:.2f}%".format(Proportion[1] / (Proportion[0] + Proportion[1]) * 100))


[<keras.engine.sequential.Sequential object at 0x0000021D4CCCF7F0>, <keras.engine.sequential.Sequential object at 0x0000021D465ED3A0>, <keras.engine.sequential.Sequential object at 0x0000021D3E612B20>, <keras.engine.sequential.Sequential object at 0x0000021D26D14520>, <keras.engine.sequential.Sequential object at 0x0000021D51BA9BB0>, <keras.engine.sequential.Sequential object at 0x0000021D5D75CBB0>, <keras.engine.sequential.Sequential object at 0x0000021D531A0B50>, <keras.engine.sequential.Sequential object at 0x0000021D259712B0>, <keras.engine.sequential.Sequential object at 0x0000021D5313ABB0>, <keras.engine.sequential.Sequential object at 0x0000021D5025ECA0>, <keras.engine.sequential.Sequential object at 0x0000021D5A2579A0>, <keras.engine.sequential.Sequential object at 0x0000021D4D15C8B0>, <keras.engine.sequential.Sequential object at 0x0000021D5ECD4FA0>, <keras.engine.sequential.Sequential object at 0x0000021D5EC890A0>, <keras.engine.sequential.Sequential object at 0x0000021D4A8F0

C:\Users\James\AppData\Local\Temp\ipykernel_18756\1740219387.py:20: RuntimeWarning: invalid value encountered in long_scalars
  precision_class_one = TP / (TP + FP)
C:\Users\James\AppData\Local\Temp\ipykernel_18756\1740219387.py:20: RuntimeWarning: invalid value encountered in long_scalars
  precision_class_one = TP / (TP + FP)


9/9 [==============================] - 0s 2ms/step


C:\Users\James\AppData\Local\Temp\ipykernel_18756\1740219387.py:20: RuntimeWarning: invalid value encountered in long_scalars
  precision_class_one = TP / (TP + FP)
C:\Users\James\AppData\Local\Temp\ipykernel_18756\1740219387.py:20: RuntimeWarning: invalid value encountered in long_scalars
  precision_class_one = TP / (TP + FP)


9/9 [==============================] - 0s 2ms/step


...

C:\Users\James\AppData\Local\Temp\ipykernel_18756\1740219387.py:20: RuntimeWarning: invalid value encountered in long_scalars
  precision_class_one = TP / (TP + FP)
C:\Users\James\AppData\Local\Temp\ipykernel_18756\1740219387.py:20: RuntimeWarning: invalid value encountered in long_scalars
  precision_class_one = TP / (TP + FP)
C:\Users\James\AppData\Local\Temp\ipykernel_18756\1740219387.py:20: RuntimeWarning: invalid value encountered in long_scalars
  precision_class_one = TP / (TP + FP)


9/9 [==============================] - 0s 2ms/step


C:\Users\James\AppData\Local\Temp\ipykernel_18756\1740219387.py:20: RuntimeWarning: invalid value encountered in long_scalars
  precision_class_one = TP / (TP + FP)


By random chance the precision would be 17.88%


C:\Users\James\AppData\Local\Temp\ipykernel_18756\1740219387.py:20: RuntimeWarning: invalid value encountered in long_scalars
  precision_class_one = TP / (TP + FP)


In [15]:
import numpy as np

# Assuming y_true and y_pred are numpy arrays representing the true labels and predicted labels, respectively.
# 1 represents the positive class, and 0 represents the negative class.
y_pred=model.predict(Dataset_test)
# Compute the confusion matrix
conf_matrix = np.zeros((2, 2), dtype=int)
conf_matrix[0, 0] = np.sum((Labels_test.values == 0) & (y_pred == 0))  # True Negatives (TN)
conf_matrix[0, 1] = np.sum((Labels_test.values== 0) & (y_pred == 1))  # False Positives (FP)
conf_matrix[1, 0] = np.sum((Labels_test.values == 1) & (y_pred == 0))  # False Negatives (FN)
conf_matrix[1, 1] = np.sum((Labels_test.values == 1) & (y_pred == 1))  # True Positives (TP)

# Extract true negatives (TN) and false positives (FP) from the confusion matrix
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
TP = conf_matrix[1, 1]
FN = conf_matrix[1, 0]
# Calculate specificity
specificity = TN / (TN + FP)
sensitivity = TP / (TP + FN)
precision_class_one=TP/(TP+FP)
Negative_Predictive_Value=TN/(TN+FN)
accuracy=(TP+TN)/(TP+TN+FP+FN)
print(f"Specificity: {specificity * 100:.2f}%",
      f"Sensitivity: {sensitivity * 100:.2f}%",
      f"Accuracy: {accuracy * 100:.2f}%",
      f"Negative Predictive Value: {Negative_Predictive_Value * 100:.2f}%",
      f"Precision (Class One): {precision_class_one * 100:.2f}%")
print(f"True Negatives (TN): {TN}",
      f"False Positives (FP): {FP}",
      f"True Positives (TP): {TP}",
      f"False Negatives (FN): {FN}")


9/9 [==============================] - 0s 2ms/step
Specificity: 99.54% Sensitivity: 6.25% Accuracy: 82.58% Negative Predictive Value: 82.69% Precision (Class One): 75.00%
True Negatives (TN): 215 False Positives (FP): 1 True Positives (TP): 3 False Negatives (FN): 45


In [ ]:
import tensorflow as tf

# Assuming `model` is your trained model
# Assuming `X_test` is your test data and `y_test` are your test labels
y_pred = DiseaseModel.predict(Dataset_test)  # assuming x_test is your test data
y_pred = (y_pred > 0.5).astype(int)
conf_matrix = np.zeros((2, 2), dtype=int)
conf_matrix[0, 0] = np.sum((Labels_test.values == 0) & (y_pred == 0))  # True Negatives (TN)
conf_matrix[0, 1] = np.sum((Labels_test.values== 0) & (y_pred == 1))  # False Positives (FP)
conf_matrix[1, 0] = np.sum((Labels_test.values == 1) & (y_pred == 0))  # False Negatives (FN)
conf_matrix[1, 1] = np.sum((Labels_test.values == 1) & (y_pred == 1))  # True Positives (TP)

# Extract true negatives (TN) and false positives (FP) from the confusion matrix
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
TP = conf_matrix[1, 1]
FN = conf_matrix[1, 0]
# Calculate specificity
specificity = TN / (TN + FP)
sensitivity = TP / (TP + FN)
precision_class_one=TP/(TP+FP)
Negative_Predictive_Value=TN/(TN+FN)
accuracy=(TP+TN)/(TP+TN+FP+FN)
print(specificity*100,sensitivity*100,accuracy*100,Negative_Predictive_Value*100,precision_class_one*100)
print(TN,FP,TP,FN)

9/9 [==============================] - 0s 4ms/step
78.66666666666666 46.93877551020408 72.99270072992701 87.192118226601 32.3943661971831
177 48 23 26
